In [33]:
import csv
import xlrd
import re
import os

working_directory=input("working directory: ")
working_directory=working_directory.replace("\\","/")
QC_summary_f=open("%s/1735-1392-SS0808-combine-need-repeat-QC summary.csv" %working_directory)
output_f=open("%s/1735-1392-SS0808-repeat-run-add-pool.csv" %working_directory,"w+")

os.chdir(working_directory)

header=QC_summary_f.readline()
sample_dict={}
for line in QC_summary_f:
    items=line.split(",")
    sample_id=items[2]
    
    plate_id=re.split(r'[-_]',sample_id)[1]
    project_id=re.split(r'[-_]',sample_id)[0]
    
    plate_id="%s-%s" %(project_id,plate_id)
    sample_reads=int(items[4])
    barcode_seq=items[3]
    compensate_reads=10000- sample_reads
    if compensate_reads<0:
        continue
    else:
        add_vol=round(compensate_reads/1000)
        if add_vol==0:
            add_vol=1
    
    if plate_id in sample_dict.keys():
        sample_dict[plate_id].append((sample_id,add_vol,barcode_seq))
    else:
        sample_dict[plate_id]=[(sample_id,add_vol,barcode_seq)]
QC_summary_f.close()
    

working directory: X:\Admera\Inhouse\Misq\1735-1392-SS0802-SS0808\1735-1392-SS0808-combine


In [34]:
#输出的csv 文件包括了sample id, well pos, add vol,i7 seq, i5 seq这些信息。
output_f.write("sample_id,well_pos,add_vol,i7_seq,i5_seq\n")
sample_dict_update={}
for key1,value1 in sample_dict.items():
    sample_dict_update[key1]=[]
    for item in value1:
        sample_id=item[0]
        add_vol=item[1]
        barcode_seq=item[2]
        i7_seq,i5_seq=barcode_seq.split("+")
        well_pos=re.split(r"[-_]",sample_id)[2]
        matched_result=re.match("([A-Z]+)(\d+)",well_pos)
        row=matched_result.group(1)
        col=int(matched_result.group(2))
        col="{:02d}".format(col)
        #print (row,col)
        well_pos=row+col
        #print (sample_id,well_pos,add_vol)
        output_f.write(",".join([sample_id,well_pos,str(add_vol),i7_seq,i5_seq,"\n"]))
        sample_dict_update[key1].append((sample_id,well_pos,str(add_vol),i7_seq,i5_seq))
    output_f.write("\n")
output_f.close()


In [35]:
sample_dict_update

{'SS0808-96PLT708': [('SS0808_96PLT708_A05',
   'A05',
   '4',
   'TCGACAAG',
   'ATATGCGC'),
  ('SS0808_96PLT708_C01', 'C01', '1', 'ACGATCAG', 'AACCGTTC'),
  ('SS0808_96PLT708_C05', 'C05', '1', 'TCGACAAG', 'AACCGTTC'),
  ('SS0808_96PLT708_C11', 'C11', '1', 'GATCTTGC', 'AACCGTTC'),
  ('SS0808_96PLT708_E07', 'E07', '1', 'ATCATGCG', 'GAACATCG'),
  ('SS0808_96PLT708_G03', 'G03', '2', 'CTAGCTCA', 'TCAGGCTT'),
  ('SS0808_96PLT708_H02', 'H02', '1', 'TCGAGAGT', 'GTTCTCGT'),
  ('SS0808_96PLT708_H05', 'H05', '1', 'TCGACAAG', 'GTTCTCGT'),
  ('SS0808_96PLT708_H09', 'H09', '2', 'ATTAGCCG', 'GTTCTCGT'),
  ('SS0808_96PLT708_H10', 'H10', '2', 'CGATCGAT', 'GTTCTCGT'),
  ('SS0808_96PLT708_H11', 'H11', '1', 'GATCTTGC', 'GTTCTCGT')],
 'SS0808-96PLT710': [('SS0808_96PLT710_B02',
   'B02',
   '1',
   'TCTTCGAC',
   'TGGTACAG'),
  ('SS0808_96PLT710_B03', 'B03', '4', 'ATCGTGGT', 'TGGTACAG'),
  ('SS0808_96PLT710_B04', 'B04', '3', 'CGGTAATC', 'TGGTACAG'),
  ('SS0808_96PLT710_B05', 'B05', '5', 'AGTTGTGC', 'TGGT

In [36]:
#自动生成auto pipette 和 pooling 文件
from openpyxl import load_workbook
import os
import csv
auto_pipette_template="C:/Users/xyy69/Documents/auto_pipette_auto_pool/Admera Pooling Template - Final Version.xlsx"



#输出auto pipette的excel 文件
for key,value in sample_dict_update.items():
    wb=load_workbook(auto_pipette_template)
    ws_samples=wb['Samples']
    sample_ID_column=2
    sample_well_column=3
    sample_ul_column=4
    
    row_start=3
    #如果1个plate 中小于10个样品要pool，就手动pool
    if len(value)<10:
        continue
    for item in value:
        
        sample_id=item[0]
        well_pos=item[1]
        add_vol=item[2]
        cell_sample_id=ws_samples.cell(row=row_start,column=sample_ID_column)
        cell_sample_id.value=sample_id
        cell_well_pos=ws_samples.cell(row=row_start,column=sample_well_column)
        cell_well_pos.value=well_pos
        cell_add_vol=ws_samples.cell(row=row_start,column=sample_ul_column)
        cell_add_vol.value=add_vol
        row_start+=1
    cell_pool_id=ws_samples.cell(row=3,column=1)
    cell_pool_id.value="1735-1392-%s-CL-pool" %key
    auto_excel="1735-1392-%s-CL-pool.xlsx" %key
    wb.save(auto_excel)
    #由于必须用excel存储才会有缓存，因此必须打开，再储存。
    os.startfile(auto_excel)  


    
    
    


In [37]:

#输出auto pipette 的csv文件
for key,value in sample_dict_update.items():
    row_start=3
    if len(value)<10:
        continue       
    auto_excel="1735-1392-%s-CL-pool.xlsx" %key
    wb_auto=load_workbook(auto_excel,data_only=True)
    ws_csv=wb_auto['CSV1']
    csv_f=open("./1735-1392-%s-CL-pool.csv" %key,"w+",newline='')
    csv_writer=csv.writer(csv_f,delimiter=",")
    for row in ws_csv.iter_rows():       
        csv_row=[]
        for cell in row:
            #print (cell.value)
            csv_row.append(cell.value)
        csv_writer.writerow(csv_row)
    csv_f.close()
    
    if len(value)>50:
        ws_csv=wb_auto['CSV2']
        csv_f=open("./1735-1392-%s-CL-pool-second.csv" %key,"w+",newline='')
        csv_writer=csv.writer(csv_f,delimiter=",")
        for row in ws_csv.iter_rows():       
            csv_row=[]
            for cell in row:
                #rint (cell.value)
                csv_row.append(cell.value)
            csv_writer.writerow(csv_row)
        csv_f.close()
        

In [38]:
#为pooling sheet 做准备
sample_dict_pooling_sheet={}
for key,value in sample_dict_update.items():
    #多于10个样品的，自动pool，否则，手动pool
    plt_id=key.split("-")[0]+'-manual'
    if len(value)>10:
        sample_dict_pooling_sheet[key]=value
    else:    
        if plt_id in sample_dict_pooling_sheet.keys():
            sample_dict_pooling_sheet[plt_id]=sample_dict_pooling_sheet[plt_id]+value
        else:
            sample_dict_pooling_sheet[plt_id]=value
            
from openpyxl.styles.borders import Border, Side
from openpyxl.styles import Alignment
from openpyxl import Workbook


from openpyxl.worksheet.dimensions import ColumnDimension, DimensionHolder
from openpyxl.utils import get_column_letter

pooling_sheet_id=input("pooling and cleaning id:")
wb_pooling=Workbook()
ws_pooling=wb_pooling.active
thin = Side(border_style="thin", color="000000")
ws_pooling.cell(row=1,column=1).value='Pool ID'
ws_pooling.cell(row=1,column=2).value='Sample ID'
ws_pooling.cell(row=1,column=3).value='Well Position'
ws_pooling.cell(row=1,column=4).value='ul to pool'
for row in ws_pooling['A1:D1' ]:
    for cell in row:
        cell.border=Border(top=thin,left=thin,right=thin,bottom=thin)

row_count=2
max_pool_id_size=0
max_sample_id_size=0
for key,value in sample_dict_pooling_sheet.items():
    pool_id='1735-1392-%s-CL-pool' %key
    if len(pool_id)>max_pool_id_size:
        max_pool_id_size=len(pool_id)
    row_start=row_count
    ws_pooling.cell(row=row_start,column=1).value=pool_id
    for item in value:
        sample_id=item[0]
        if len(sample_id)>max_sample_id_size:
            max_sample_id_size=len(sample_id)
        well_pos=item[1]
        add_vol=item[2]
        ws_pooling.cell(row=row_count,column=2).value=sample_id
        ws_pooling.cell(row=row_count,column=3).value=well_pos
        ws_pooling.cell(row=row_count,column=4).value=add_vol
        row_count+=1
    row_end=row_count-1
    
#调整格式    
    ws_pooling.merge_cells('A%s:A%s' %(row_start,row_end))
    for row in ws_pooling['A%s:D%s' %(row_start,row_end)]:
        for cell in row:
            cell.border=Border(top=thin,left=thin,right=thin,bottom=thin)
    ws_pooling.cell(row=row_start,column=1).alignment=Alignment(horizontal='center',vertical='center')
    
    row_count+=1

ws_pooling.column_dimensions['A'].width = int(max_pool_id_size*1.2)
ws_pooling.column_dimensions['B'].width = int(max_sample_id_size*1.2)
ws_pooling.column_dimensions['C'].width = 15
ws_pooling.column_dimensions['D'].width = 11

'''
dim_holder = DimensionHolder(worksheet=ws_pooling)
for col in range(ws_pooling.min_column, ws_pooling.max_column + 1):
    dim_holder[get_column_letter(col)] = ColumnDimension(ws_pooling, min=col, max=col, width=20)
ws_pooling.column_dimensions=dim_holder
'''
wb_pooling.save('1735-1392-%s-pooling-and-cleaning.xlsx' %pooling_sheet_id)
    
    
    
        
    

pooling and cleaning id:1735-1392-SS0808


In [17]:
sample_dict_update

{'SS0748-PLT511': [('SS0748_PLT511_A02', 'A02', '1', 'TCGAGAGT', 'ATATGCGC'),
  ('SS0748_PLT511_B09', 'B09', '2', 'ATTAGCCG', 'TGGTACAG'),
  ('SS0748_PLT511_C02', 'C02', '1', 'TCGAGAGT', 'AACCGTTC'),
  ('SS0748_PLT511_C06', 'C06', '1', 'CCTTGGAA', 'AACCGTTC'),
  ('SS0748_PLT511_C07', 'C07', '6', 'ATCATGCG', 'AACCGTTC'),
  ('SS0748_PLT511_C08', 'C08', '3', 'TGTTCCGT', 'AACCGTTC'),
  ('SS0748_PLT511_D06', 'D06', '3', 'CCTTGGAA', 'TAACCGGT'),
  ('SS0748_PLT511_D07', 'D07', '5', 'ATCATGCG', 'TAACCGGT'),
  ('SS0748_PLT511_D08', 'D08', '6', 'TGTTCCGT', 'TAACCGGT'),
  ('SS0748_PLT511_D09', 'D09', '5', 'ATTAGCCG', 'TAACCGGT'),
  ('SS0748_PLT511_E03', 'E03', '2', 'CTAGCTCA', 'GAACATCG'),
  ('SS0748_PLT511_E04', 'E04', '1', 'ATCGTCTC', 'GAACATCG'),
  ('SS0748_PLT511_E05', 'E05', '6', 'TCGACAAG', 'GAACATCG'),
  ('SS0748_PLT511_E06', 'E06', '4', 'CCTTGGAA', 'GAACATCG'),
  ('SS0748_PLT511_E08', 'E08', '1', 'TGTTCCGT', 'GAACATCG'),
  ('SS0748_PLT511_E09', 'E09', '1', 'ATTAGCCG', 'GAACATCG'),
  ('SS0

In [39]:
#make samplesheet
sample_sheet_f=open('C:/Users/xyy69/Documents/auto_pipette_auto_pool/SampleSheet_16s_template.csv')
sample_sheet_id=input('sample_sheet_id:')
sample_sheet_output_f=open("%s-SampleSheet.csv" %sample_sheet_id,"w+")
for line in sample_sheet_f:
    if line.startswith("Experiment-Name"):
        line=line.replace("16s","%s" %sample_sheet_id)
    sample_sheet_output_f.write(line)
for plt_id,sample_list in sample_dict_update.items():
    for item in sample_list:
        sample_id=item[0]
        i7_index=item[3]
        i5_index=item[4]
        sample_sheet_output_f.write(",".join([sample_id,'','','','',i7_index,'',i5_index,'','\n']))
        
sample_sheet_output_f.close()


sample_sheet_id:1735-1392-SS0808-repeat
